# Export a shapefile of a general MODFLOW model from the NHDPlus dataset

This notebook implements a grid-search approach to finding hydraulic conductivities that result in heads that seem reasonable.

Project specific variables are imported in the model_spec.py and gen_mod_dict.py files that must be included in the notebook directory. The first first includes pathnames to data sources that will be different for each user. The second file includes a dictionary of model-specific information such as  cell size, default hydraulic parameter values, and scenario defintion (e.g. include bedrock, number of layers, etc.). There are examples in the repository. Run the following cells up to the "Run to here" cell to get a pull-down menu of models in the model_dict. Then, without re-running that cell, run all the remaining cells.  Re-running the following cell would re-set the model to the first one in the list, which you probably don't want. If you use the notebook option to run all cells below, it runs the cell you're in, so if you use that option, move to the next cell (below the pull-down menu of models) first.

In [1]:
__author__ = 'Jeff Starn'
%matplotlib notebook
from model_specs import *
from gen_mod_dict import *

import os, sys
import shutil
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import gdal
gdal.UseExceptions()

from flopy.utils.postprocessing import get_water_table


import flopy.utils.binaryfile as bf
from matplotlib import colors
import flopy as fp
import pandas as pd
# import ipyparallel as ipp
# from model_specs import *
# from gen_mod_dict import *

from ipywidgets import interact, Dropdown
from IPython.display import display

print(sys.version)

3.5.3 |Continuum Analytics, Inc.| (default, Feb 22 2017, 21:28:42) [MSC v.1900 64 bit (AMD64)]


The next cell is a template for making this notebook into a batch script. To do so, save this notebook as a .py file and edit it as follows. Comment out all the notebook-specific commands (drop-down menu stuff and commands preceded by %). Indent everything below the next cell twice so that it falls within the 'for' loop and the 'try' statement. Move the 'except' statement to the end of the script. Comment out lines in the cell after 'Preliminary stuff' so that the model is selected in the 'for' loop from gen_mod_dict. You can leave the print statement in that cell uncommented. 

In [2]:
for key, value in model_dict.items():   # from "gen_mod_dict.py"
    md = key
    ms = model_dict[md]
    print('trying {}'.format(md))
    try:
        pass
    except:
        pass

trying Niantic250
trying CoastalCT250
trying CoastalCT500
trying Neversink
trying Niantic500
trying CoastalCT
trying Assabet
trying Niantic


In [3]:
models = list(model_dict.keys())
models.sort()
model_area = Dropdown(
    options=models,
    description='Model:',
    background_color='cyan',
    border_color='black',
    border_width=2)
display(model_area)

### Run to here to initiate notebook

First time using this notebook in this session (before restarting the notebook), run the cells up to this point. Then select your model from the dropdown list above. Move your cursor to this cell and use the toolbar menu Cell --> Run All Below.  After the first time, if you want to run another model, select your model and start running from this cell--you don't need to re-run the cells from the beginning.

## Preliminary stuff

In [4]:
md = model_area.value
ms = model_dict[md]
print('The model being processed is {}\n'.format(md))


if 'L' in ms.keys():
    L = ms['L']
print('The resolution is %0.2f m\n'%(L))

The model being processed is CoastalCT500

The resolution is 152.40 m



### Get the calibration scenario

Copy model to be calibrated (parent model) to new directory. Define the weight, which is used after all the models are run to find the best set of parameters by weighting the error rates. It is specified here so it can be used in the directory name. Weights > 1 result in fewer dry drains and more cells with heads above land surface. That tends to mean lower hydraulic conductivities.

In [5]:
hydro_wt = 1.0
calib=False
PEST=False
label = "DRN0_smoothDrn_GHBCorrections_PEST - Copy"
label="DRN0_PEST__WellDateAdjust_RIV"
label="DRN0_PEST"
label="SFR_DRN0_PEST"
# label="DRN0"
# label="DRN_cal_wt_1.00"

geo_ws = os.path.join(proj_dir, ms['ws'])
parent_ws = os.path.join(geo_ws, scenario_dir)
if calib:
    dir_name = '{}_cal_wt_{:4.2f}'.format(scenario_dir, hydro_wt)
else:
    dir_name = scenario_dir
    
if PEST:
    dir_name = dir_name + "_PEST"
    
if label!='':
    dir_name = dir_name +"_"+str(label)
    
model_ws = os.path.join(geo_ws, dir_name)

if not os.path.exists(model_ws):
    print('This calibration scenario has not been completed')

In [6]:
def make_raster(dst_file, data, NCOL, NROW, gt, proj, nodata):
    '''
    Writes numpy array to a GeoTiff file.
    
    dst_file : name of file to write
    data : 2D numpy array
    NCOL, NROW : number of rows and columns. These may coincide with a MODFLOW grid.
    gt : 6-element geotransform list [C, A, B, F, E, D]. Gives the coordinates of one pixel
        (the upper left pixel). If there is no rotation, B=D=0. If cells are square, A=-E.   
        Letter designations come from the original documentation.
        
        C = x coordinate in map units of the upper left corner of the upper left pixel
        A = distance from C along x axis to upper right pixel corner of the upper left pixel
        B = distance from C along x axis to lower left pixel corner of the upper left pixel,
        F = y coordinate in map units of the upper left corner of the upper left pixel
        E = distance from C along y axis to lower left pixel corner of the upper left pixel
        D = distance from C along y axis to upper right pixel corner of the upper left pixel
        
    proj : projection of the GeoTiff
    nodata : value to use as missing data in the GeoTiff
    '''
    import gdal
    driver = gdal.GetDriverByName("GTiff")
    dst = driver.Create(dst_file, NCOL, NROW, 1, gdal.GDT_Float32)
    dst.SetGeoTransform(gt)
    dst.SetProjection(proj)
    band = dst.GetRasterBand(1)
    band.SetNoDataValue(nodata)
    band.WriteArray(data)
    dst = None

Load existing model and some packages needed for parameter estimation

In [7]:
nam_file = '{}.nam'.format(md)
mf = fp.modflow.Modflow.load(os.path.join(model_ws,nam_file), version='mfnwt', exe_name=mfpth, verbose=False, model_ws=model_ws, load_only=None)

bas = mf.get_package('BAS6')
dis = mf.get_package('DIS')
upw = mf.get_package('UPW')
oc = mf.get_package('OC')
well = mf.get_package('WEL')
rch = mf.get_package('RCH')
head_file_pth = os.path.join(model_ws, '{}.hds'.format(md))

ibound = bas.ibound
botm = dis.getbotm()
hdsobj = bf.HeadFile(head_file_pth)
hds = hdsobj.get_data()
wt = get_water_table(heads=hds, nodata=-8890)

model_file = os.path.join(geo_ws, dir_name,'model_grid.csv')
model_grid = pd.read_csv(model_file, na_values=[hnoflo, hdry])

land_surface = model_grid.top
top = land_surface.values.reshape(dis.nrow, dis.ncol)
node = model_grid.node_num.values.reshape(dis.nrow,dis.ncol)

In [8]:
#convert obs type to numeric values
model_grid['obs_type_int'] = 0
model_grid.loc[model_grid.obs_type=="hydro",'obs_type_int']=1
model_grid.loc[model_grid.obs_type=="topo","obs_type_int"]=2

In [9]:
#make a dictionary of heads to export with the model parameters
other_dict={}

#loop through the model layers
for i in range(hds.shape[0]):
    other_dict["head_"+str(i)]=hds[i,:,:]

    zone_file = os.path.join(model_ws,'zone_array_detailed.npz')    
if not os.path.isfile(zone_file):
    zone_file = os.path.join(model_ws, 'zone_array.npz')
    
if os.path.isfile(zone_file):
    zones = np.load(zone_file)
    zones = zones['zone']
    for i in range(zones.shape[0]):
        other_dict["surfzone_"+str(i)]=zones[i,:,:]

other_dict["ZoneBud"]=model_grid.HUC12_shortCD.values.reshape(dis.nrow, dis.ncol)
try:
    other_dict["Septic_flux"]=model_grid.Septic_flux.values.reshape(dis.nrow, dis.ncol)
    other_dict["sep_cm_yr"]=other_dict["Septic_flux"]/L/L*365*100  #converts m3/d to cm/yr
    model_grid['PrivWell_flux']=model_grid.population.values*np.abs(model_grid.pws.values-1)*useResPerCap
    model_grid['NetPrivSeptLoss'] = model_grid.population.values*np.abs(model_grid.pws.values-1)*useResPerCap-model_grid.Septic_flux.values
    other_dict['PrivWell_flux']=model_grid.population.values.reshape(dis.nrow, dis.ncol)*np.abs(model_grid.pws.values.reshape(dis.nrow, dis.ncol)-1)*useResPerCap
    other_dict['PriWe_cm_yr']=other_dict['PrivWell_flux']/L/L*365*100  #converts m3/d to cm/yr
    model_grid['NetPrivSeptLoss'] = model_grid.population.values*np.abs(model_grid.pws.values-1)*useResPerCap-model_grid.Septic_flux.values
    other_dict['NetPrivSeptLoss']= model_grid.NetPrivSeptLoss.values.reshape(dis.nrow,dis.ncol)
    other_dict['NetSepLoss_cm_yr']=other_dict['NetPrivSeptLoss']/L/L*365*100  #converts m3/d to cm/yr
    other_dict["Pop"]=model_grid.population.values.reshape(dis.nrow, dis.ncol)
    other_dict["Coast_Deep"]=model_grid.Coast_Deep.values.reshape(dis.nrow, dis.ncol)
    other_dict["Coast_Shallow"]=model_grid.Coast_Shallow.values.reshape(dis.nrow, dis.ncol)
    other_dict["ghb_sea"]=model_grid.ghb_sea.values.reshape(dis.nrow, dis.ncol)
except:
    pass
other_dict["arbolateSu"]=model_grid.arbolateSu.values.reshape(dis.nrow, dis.ncol)
try:
    other_dict["SurfGeo"]=model_grid.SurfGeo.values.reshape(dis.nrow, dis.ncol)
    other_dict["BedLithSimp"]=model_grid.BedLithSimp.values.reshape(dis.nrow, dis.ncol)
    other_dict["PerWater"]=model_grid.PerWater.values.reshape(dis.nrow, dis.ncol)
except:
    pass

other_dict["ObsType"]=model_grid.obs_type_int.values.reshape(dis.nrow, dis.ncol)
other_dict["WetlandPer"]=model_grid.WetlandPer.values.reshape(dis.nrow, dis.ncol)
other_dict["reachcode"]=model_grid.reachcode.values.reshape(dis.nrow,dis.ncol)
other_dict["nlcd"]=model_grid.nlcd.values.reshape(dis.nrow, dis.ncol)
other_dict["obs_type"]=np.array([1 if x=="hydro"else 2 if x=="topo" else 0 for x in model_grid.obs_type.values]).reshape(dis.nrow, dis.ncol)
try:
    other_dict["HUC12_shortCD"]=model_grid.HUC12_shortCD.values.reshape(dis.nrow, dis.ncol)
    other_dict["baseflowGage"]=model_grid.baseflowGage.values.reshape(dis.nrow, dis.ncol)
    other_dict["HUC12_shortCD_ext"]=model_grid.HUC12_shortCD_ext.values.reshape(dis.nrow, dis.ncol)
except:
    pass
try:
    other_dict["Coast_major"]=model_grid.Coast_major.values.reshape(dis.nrow, dis.ncol)
except:
    pass
try:
    other_dict["Pop_raw"]=model_grid.population_float.values.reshape(dis.nrow, dis.ncol)
except:
    pass
# other_dict["PerWater"]=model_grid.PerWater.values.reshape(dis.nrow, dis.ncol)
other_dict["wt"]=wt
model_grid['wt'] = wt.ravel() 
other_dict["DTW_m"]=top - wt
other_dict["land_surface"]=top
other_dict["node"]=node
try:
    other_dict["Rech_noSep"]=rch.rech.array[0][0]-other_dict["Septic_flux"]/L/L
    other_dict['Rech_nat_cm_yr']=other_dict["Rech_noSep"]*100*365
except:
    pass


In [10]:
print(np.percentile(model_grid.NetPrivSeptLoss[(model_grid.ibound==1)&(model_grid.NetPrivSeptLoss>0)],[0,25,50,75,100]))
print(np.percentile(model_grid.NetPrivSeptLoss[(model_grid.ibound==1)&(model_grid.NetPrivSeptLoss<0)],[0,25,50,75,100]))


[  0.08102106   0.13796157   0.21572173   0.39339516  37.74043213]
[ -6.13814540e+01  -1.57244831e+00  -1.04545610e+00  -6.89982050e-01
  -1.91558939e-02]


In [11]:
model_grid.columns

Index(['node_num', 'node_num.1', 'node_num.1.1', 'ibound', 'SHEDS',
       'arbolateSu', 'gess_poly', 'lake', 'kauffman_CTquat_thk',
       'HUC12_shortCD', 'HUC12_shortCD_ext', 'sewer', 'pws', 'well_PWSFlux',
       'well_Glacial', 'BedLithSimp', 'SurfGeo', 'embayment', 'baseflowGage',
       'ned', 'ned_mean', 'kauffman_bedrock_el', 'slope', 'ned_coast_min',
       'catchment', 'soller_thk', 'soller_bedrock_el', 'nlcd', 'clear',
       'clear_per_grass', 'nlcd_ag', 'WetlandPer', 'PerDev', 'PerImp',
       'population', 'rch_eff_m_SWB_NAWQA', 'rch_eff_m_Reitz_2013',
       'rch_m_Wolock', 'Coast_Deep', 'Coast_Shallow', 'PerWater', 'PerSand',
       'PerClay', 'PerSilt', 'NDep_dry', 'NDep_wet', 'edge', 'ghb', 'stage',
       'segment_len', 'order', 'arbolateSum', 'reachcode', 'reach_intermit',
       'reach_len', 'reach_int', 'riv_coast', 'road', 'road_length', 'ghb_sea',
       'fresh_head', 'Coast_all', 'Coast_major', 'top', 'lay', 'row', 'col',
       'obs_type', 'xc', 'yc', 'popula

In [12]:
model_grid.loc[(np.abs(model_grid.NetPrivSeptLoss)>20)&(model_grid.ibound==1),["population_float","population","pws","sewer","NetPrivSeptLoss","Septic_flux","PrivWell_flux"]]

,population_float,population,pws,sewer,NetPrivSeptLoss,Septic_flux,PrivWell_flux
649056,136.898987,136.898987,0.0,1.0,24.641818,0.000000,24.641818
650547,115.917328,115.917328,0.0,1.0,20.865119,0.000000,20.865119
713152,114.050552,114.050552,0.0,1.0,20.529099,0.000000,20.529099
740020,142.206085,142.206085,0.0,1.0,25.597095,0.000000,25.597095
741511,148.715134,148.715134,0.0,1.0,26.768724,0.000000,26.768724
744497,146.926743,146.926743,0.0,1.0,26.446814,0.000000,26.446814
745988,162.023560,162.023560,0.0,1.0,29.164241,0.000000,29.164241
760837,206.675262,206.675262,0.0,1.0,37.201547,0.000000,37.201547
762327,206.675262,209.669067,0.0,1.0,37.740432,0.000000,37.740432
762328,155.006439,155.006439,0.0,1.0,27.901159,0.000000,27.901159


In [13]:
#read in the model grid with travel times, if it exits
if os.path.isfile(model_file.replace("model_grid","model_grid2")):
    model_grid2 = pd.read_csv(model_file.replace("model_grid","model_grid2"), na_values=[hnoflo, hdry])
    other_dict['travTimeYrsDis']=model_grid2.medTimeOut_yrs.values.reshape(dis.nrow,dis.ncol)
    other_dict['travTimeYrsRe']=model_grid2.medTimeIn_yrs.values.reshape(dis.nrow,dis.ncol)

Extract an array of reduced pumping wells (if applicable) and extremely thin cells (that were turned inactive)

In [14]:
#check if any wells had reduced pumping
listFile = os.path.join(model_ws,"%s.list"%md)
o_file = open(listFile)
reducedPumping = False
wellList = False
colList = ""
wellDFList = []
thinList = []
for line in o_file:
    if "WELLS WITH REDUCED PUMPING FOR STRESS PERIOD" in line:
        reducedPumping = True
        wellList = True
        continue
    #this gets the end of the list of wells
    if reducedPumping and len(line[:-1].strip())==0 and wellList:
        wellList = False
    if reducedPumping and wellList:
        if colList =="":
            colList = line[:-1].strip().split()
            colList = [x.replace(".","") for x in colList]
            colList = [x.replace("-","") for x in colList]
        else:
            wellDFList.append(line[:-1].strip().split())
    if len(line[:-1].strip())>0:
        if line[:-1].strip().split()[0]=="Extremely":
            thinCol = line[:-1].strip().split()[6]
            thinRow = line[:-1].strip().split()[10]
            thinList.append([int(thinRow)-1, int(thinCol)-1])
            
o_file.close()



In [15]:
if reducedPumping:
    wellDF = pd.DataFrame(wellDFList,columns=colList)
    wellArray = np.zeros((dis.nrow, dis.ncol))
    
    #add the wells to the array
    for thisRow in wellDF.itertuples():
        row = int(getattr(thisRow, "ROW"))-1
        col = int(getattr(thisRow,"COL"))-1
        thisReduced = (float(getattr(thisRow,"APPLQ"))-float(getattr(thisRow,"ACTQ")))/float(getattr(thisRow,"APPLQ"))*100.0
        wellArray[row,col]=thisReduced
    
    other_dict["pumpingReduction"]=wellArray
if len(thinList)>0:
    cellArray = np.zeros((dis.nrow, dis.ncol))
    #add the wells to the array
    for thisCell in thinList:
        row = thisCell[0]
        col = thisCell[1]
        cellArray[row,col]=1
    
    other_dict["thinCells"]=cellArray

In [16]:
cbb = fp.utils.CellBudgetFile(os.path.join(model_ws,md+".cbc"))
try:
    CGWD = cbb.get_data(text=b' HEAD DEP BOUNDS')[0]
    CGWD_DF = pd.DataFrame(CGWD)
    CGWD_DF = CGWD_DF.rename(index=str,columns={"node":"node_num"})
    #adjust the node numbering
    CGWD_DF.node_num = CGWD_DF.node_num-1
    model_grid = pd.merge(model_grid,CGWD_DF,"left","node_num", sort=False)
    CGWD_q = -1*model_grid.q.values.reshape(dis.nrow, dis.ncol)
    other_dict['CGWD_q'] = CGWD_q
except:
    pass


#outFile = os.path.join(model_ws,"CGWD_out2.csv")
#np.savetxt(outFile,CGWD,delimiter=',', header=','.join(CGWD.dtype.names), comments="")


In [17]:
if b'          DRAINS' in cbb.get_unique_record_names():
    Drn = cbb.get_data(text=b' DRAINS')[0]
    Drn_DF = pd.DataFrame(Drn)
    Drn_DF = Drn_DF.rename(index=str,columns={"node":"node_num"})
    #adjust the node numbering
    Drn_DF.node_num = Drn_DF.node_num-1
    model_grid = pd.merge(model_grid,Drn_DF,"left","node_num", sort=False)
    if "q_y" in model_grid.columns:
        Drn_q = -1*model_grid.q_y.values.reshape(dis.nrow, dis.ncol)
    else:
        Drn_q = -1*model_grid.q.values.reshape(dis.nrow, dis.ncol)
    other_dict['DRN_q'] = Drn_q


    # fname = '{}.tif'.format('DRN_q')
    # dst = os.path.join(model_ws, fname)
    # if os.path.exists(dst):
    #     os.remove(dst)
    # data = DRN_q.values.reshape(dis.nrow,dis.ncol)
    # make_raster(dst, data, dis.ncol, dis.nrow, gt, shapeproj, np.nan)


    #outFile = os.path.join(model_ws,"CGWD_out2.csv")
    #np.savetxt(outFile,CGWD,delimiter=',', header=','.join(CGWD.dtype.names), comments="")


In [18]:
if b'          DRAINS' in cbb.get_unique_record_names():
    fname = '{}.tif'.format('DRN_q')
    dst = os.path.join(model_ws, fname)
    if os.path.exists(dst):
        os.remove(dst)
    thisDataTemp = other_dict['DRN_q'].reshape(dis.nrow,dis.ncol)
    thisDataTemp[np.isnan(thisDataTemp)]=0
    thisData = os.path.join(model_ws,"DRN_q.dat")
    np.savetxt(thisData, thisDataTemp, fmt='%15.6E', delimiter='')

    commandTxt = '"C:\\Users\\jbarclay\\.conda\\envs\\GenMod\\python" "C:\\Users\\jbarclay\\OneDrive - DOI\\LISS\\Analysis\\Python\\ArrayToTiff.py" %s "%s" %s %s'%(md,model_ws,thisData,dst)

    info = subprocess.STARTUPINFO()
    info.dwFlags = subprocess.STARTF_USESHOWWINDOW
    info.wShowWindow = subprocess.SW_HIDE

    with subprocess.Popen(commandTxt, stdout=subprocess.PIPE, startupinfo=info, stderr = subprocess.STDOUT, bufsize=1, universal_newlines=True) as p:

        for line in p.stdout:
            print(line, end='')


In [19]:
if b'   RIVER LEAKAGE' in cbb.get_unique_record_names():
    RIV = cbb.get_data(text=b'   RIVER LEAKAGE')[0]
    RIV_DF = pd.DataFrame(RIV)
    RIV_DF = RIV_DF.rename(index=str,columns={"node":"node_num"})
    #adjust the node numbering
    RIV_DF.node_num = RIV_DF.node_num-1
    RIV_DF.rename(columns={'q':"RIV_q"}, inplace=True)
    model_grid = pd.merge(model_grid,RIV_DF,"left","node_num", sort=False)
    RIV_q = -1*model_grid.RIV_q.values.reshape(dis.nrow, dis.ncol)
    other_dict['RIV_q'] = RIV_q


#outFile = os.path.join(model_ws,"CGWD_out2.csv")
#np.savetxt(outFile,CGWD,delimiter=',', header=','.join(CGWD.dtype.names), comments="")


In [20]:
if b'  STREAM LEAKAGE' in cbb.get_unique_record_names():
    sfr = mf.get_package('SFR')
    SFR = cbb.get_data(text=b'  STREAM LEAKAGE')[0]
    SFR_DF = pd.DataFrame(SFR)
    SFR_DF = SFR_DF.rename(index=str,columns={"node":"node_num"})
    #flatten the nodes to 1 layer
    while np.max(SFR_DF.node_num)>(dis.nrow*dis.ncol):
        SFR_DF.loc[SFR_DF.node_num>(dis.nrow*dis.ncol),"node_num"] = SFR_DF.loc[SFR_DF.node_num>(dis.nrow*dis.ncol),"node_num"] - (dis.nrow*dis.ncol)
    
    #adjust the node numbering
    SFR_DF.node_num = SFR_DF.node_num-1
    SFR_DF.rename(columns={'q':"SFR_q"}, inplace=True)
    SFR_DF = SFR_DF.groupby(["node_num"],as_index=False)["SFR_q"].sum()
    model_grid = pd.merge(model_grid,SFR_DF,"left","node_num", sort=False)
    SFR_q = -1*model_grid.SFR_q.values.reshape(dis.nrow, dis.ncol)
    other_dict['SFR_q'] = SFR_q


#outFile = os.path.join(model_ws,"CGWD_out2.csv")
#np.savetxt(outFile,CGWD,delimiter=',', header=','.join(CGWD.dtype.names), comments="")


In [21]:
#add the combined well flux to the model_grid
#get the well fluxes (layers 3 & 4)
pumpFluxes = well.stress_period_data.df[well.stress_period_data.df.k>1]

#adjust the node numbering
pumpFluxes.node = pumpFluxes.node

#aggregate by cell
pumpFluxes = pumpFluxes.groupby('node').sum()
pumpFluxes.reset_index(inplace=True)
model_grid = pd.merge(model_grid,pumpFluxes,"left",left_on="node_num",right_on="node",sort=False)

Well_Flux = model_grid.flux0.values.reshape(dis.nrow,dis.ncol)
other_dict['Well_Flux']=Well_Flux

In [22]:
#open the grid specs file to get the xul and yul values
grid_specs_file = os.path.join(geo_ws,"grid_spec.txt")

o_file = open(grid_specs_file)
marker = 0
for line in o_file:
    if marker=="ul":
        coords = line[:-1].split()
        marker=0
    if marker=="rotation":
        rotation = float(line[:-1].split()[1])
        marker=0
    if len(line)>3:
        if line.split()[0]=="Upper" and line.split()[1]=="left":
            marker="ul"
        if line.split()[0]=="Rotation":
            marker = "rotation"
o_file.close()

xul = float(coords[0])
yul = float(coords[1])

In [23]:
#set the spatial reference

mf.sr = fp.utils.reference.SpatialReference(delr=[L]*dis.ncol, delc=[L]*dis.nrow, lenuni=dis.lenuni, xul = xul, yul = yul, rotation = rotation, units="meters", proj4_str="EPSG:5070")

#use the prj file from the domain file
prj = md+"_domain.prj"
prj = os.path.join(geo_ws,prj)
#temporarily change the directory
thisDir = os.getcwd()
os.chdir(model_ws)
fp.export.shapefile_utils.model_attributes_to_shapefile(md+".shp",mf,array_dict=other_dict, prj=prj)

#change the directory back
os.chdir(thisDir)

wrote CoastalCT500.shp


In [24]:
noFlowDF = model_grid.loc[(model_grid.ibound==1)&(model_grid.q_y==0)&(model_grid.ghb_sea==1)&(model_grid.reachcode!=0),["q_y","reachcode","arbolateSu","lake","Coast_Deep","Coast_Shallow","fresh_head","wt"]]

In [25]:
noFlowDF['headDiff']=noFlowDF.fresh_head-noFlowDF.wt
np.percentile(noFlowDF.headDiff,[0,25,50,75,100])

array([  1.57855253e-03,   1.70962900e-01,   5.90032578e-01,
         2.06264549e+00,   1.29283943e+01])

In [26]:
model_grid.columns

Index(['node_num', 'node_num.1', 'node_num.1.1', 'ibound', 'SHEDS',
       'arbolateSu', 'gess_poly', 'lake', 'kauffman_CTquat_thk',
       'HUC12_shortCD', 'HUC12_shortCD_ext', 'sewer', 'pws', 'well_PWSFlux',
       'well_Glacial', 'BedLithSimp', 'SurfGeo', 'embayment', 'baseflowGage',
       'ned', 'ned_mean', 'kauffman_bedrock_el', 'slope', 'ned_coast_min',
       'catchment', 'soller_thk', 'soller_bedrock_el', 'nlcd', 'clear',
       'clear_per_grass', 'nlcd_ag', 'WetlandPer', 'PerDev', 'PerImp',
       'population', 'rch_eff_m_SWB_NAWQA', 'rch_eff_m_Reitz_2013',
       'rch_m_Wolock', 'Coast_Deep', 'Coast_Shallow', 'PerWater', 'PerSand',
       'PerClay', 'PerSilt', 'NDep_dry', 'NDep_wet', 'edge', 'ghb', 'stage',
       'segment_len', 'order', 'arbolateSum', 'reachcode', 'reach_intermit',
       'reach_len', 'reach_int', 'riv_coast', 'road', 'road_length', 'ghb_sea',
       'fresh_head', 'Coast_all', 'Coast_major', 'top', 'lay', 'row', 'col',
       'obs_type', 'xc', 'yc', 'popula

In [27]:
#mean recharge for cells with and without discharge
RCH = rch.rech.array[0][0]-other_dict["Septic_flux"]/L/L
print(np.percentile(RCH[ibound.array[0]==1],[25,50,75]))
print(np.percentile(RCH[(ibound.array[0]==1)&((CGWD_q>0)|(Drn_q>0))],[25,50,75]))
print(np.percentile(RCH[(ibound.array[0]==1)&((CGWD_q<=0)&(Drn_q<=0))],[25,50,75]))
COAST = model_grid.Coast_major.values.reshape(dis.nrow, dis.ncol)

[ 0.00070479  0.00082496  0.000939  ]
[ 0.          0.00078197  0.00091578]
[ 0.00072124  0.00082911  0.00094275]


In [28]:
CGWD_q.shape

(672, 1491)

In [29]:
print("Drainage cell fraction of all cells")
print(np.sum([(ibound.array[0]==1)&((CGWD_q>0)|(Drn_q>0))])/np.sum([ibound.array[0]]))
print("Drainage cell fraction of cells non coastal-major cells")
print(np.sum([(ibound.array[0]==1)&((CGWD_q>0)|(Drn_q>0))&(COAST==0)])/np.sum([ibound.array[0][COAST==0]]))
print("Drainage cell fraction of non-septic recharge")
print(np.sum(RCH[(ibound.array[0]==1)&((CGWD_q>0)|(Drn_q>0))&(COAST==0)])/np.sum(RCH[(ibound.array[0]==1)&(COAST==0)]))
print("Drainage cell fraction of all recharge")
print(np.sum(rch.rech.array[0][0][(ibound.array[0]==1)&((CGWD_q>0)|(Drn_q>0))&(COAST==0)])/np.sum(rch.rech.array[0][0][(ibound.array[0]==1)&(COAST==0)]))

Drainage cell fraction of all cells
0.129227753665
Drainage cell fraction of cells non coastal-major cells
0.102912505728
Drainage cell fraction of non-septic recharge
0.0951508985456
Drainage cell fraction of all recharge
0.094764


In [30]:
print("Within the Niantic Watershed")
Niantic = model_grid.Niantic.values.reshape(dis.nrow, dis.ncol)
print("Drainage cell fraction of cells")
print(np.sum([(Niantic==1)&(ibound.array[0]==1)&((CGWD_q>0)|(Drn_q>0))])/np.sum([(Niantic==1)&(ibound.array[0]==1)]))
print("Drainage cell fraction of non-septic recharge")
print(np.sum(RCH[(Niantic==1)&(ibound.array[0]==1)&((CGWD_q>0)|(Drn_q>0))])/np.sum(RCH[(Niantic==1)&(ibound.array[0]==1)]))
print("Drainage cell fraction of all recharge")
print(np.sum(rch.rech.array[0][0][(Niantic==1)&(ibound.array[0]==1)&((CGWD_q>0)|(Drn_q>0))])/np.sum(rch.rech.array[0][0][(Niantic==1)&(ibound.array[0]==1)]))

Within the Niantic Watershed


AttributeError: 'DataFrame' object has no attribute 'Niantic'

In [ ]:
model_grid.columns

In [ ]:
model_grid.loc[(model_grid.ibound==1) & (pd.notnull(model_grid.stage)) & (model_grid.q_y==0),["order",'q_y']].groupby("order").count()

In [ ]:
np.unique(model_grid.stage[pd.notnull(model_grid.stage)])

In [ ]:
239/817

In [ ]:
for f in other_dict.keys():
    print(f)